<a href="https://colab.research.google.com/github/shickselate/calliope-dev/blob/main/FaceToBrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python mediapipe matplotlib nibabel scikit-image plotly

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from google.colab import files
from pathlib import Path


In [ ]:
import nibabel as nib
from skimage import measure

In [ ]:
uploaded = files.upload()
image_path = Path(next(iter(uploaded)))
image = cv2.imread(str(image_path))
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
h, w, _ = image.shape

# plt.imshow(rgb_image)
# plt.title("Uploaded Face")
# plt.axis('off')
# plt.show()


In [ ]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)
results = face_mesh.process(rgb_image)

landmarks = []
if results.multi_face_landmarks:
    for lm in results.multi_face_landmarks[0].landmark:
        x, y = int(lm.x * w), int(lm.y * h)
        landmarks.append((x, y))

landmarks = np.array(landmarks)
print(f"Extracted {len(landmarks)} facial landmarks.")

# Visualise landmarks
image_copy = rgb_image.copy()
for (x, y) in landmarks:
    cv2.circle(image_copy, (x, y), 10, (0, 255, 0), -1)

plt.figure(figsize=(8, 8))
plt.imshow(image_copy)
plt.title("Facial Landmarks")
plt.axis('off')
plt.show()


In [ ]:
# Approximate MediaPipe indices
fiducial_indices = [168, 33, 263, 234, 454]  # nasion, LOC, ROC, LPA, RPA
subject_fiducials_2d = landmarks[fiducial_indices]
print("Subject facial fiducials:\n", subject_fiducials_2d)


Subject facial fiducials:
 [[1257 1419]
 [ 844 1465]
 [1615 1465]
 [ 622 1645]
 [1748 1631]]


In [ ]:
import numpy as np
import plotly.graph_objs as go

# ------------------------
# Configuration
# ------------------------

FIDUCIAL_NAMES = ["Nasion", "LOC", "ROC", "LPA", "RPA"]
TARGET_NAMES = ["ACC", "PGC", "SGC"]


TEMPLATE_FIDUCIALS = np.array([
    [0, 100, 5],     # Nasion — forward and higher
    [-30, 95, 0],    # LOC — eye level
    [30, 95, 0],     # ROC
    [-60, 50, -15],  # LPA — tragus region
    [60, 50, -15]    # RPA
])


TEMPLATE_TARGETS = np.array([
    [0, 50, 30],     # ACC
    [0, 40, 0],      # Pregenual ACC
    [0, 25, -10],    # Subgenual ACC
    [-22, -4, -20],  # Left Amygdala
    [22, -4, -20],   # Right Amygdala
    [-12, 10, 5],    # Left ALIC
    [12, 10, 5]      # Right ALIC
])

TARGET_NAMES = [
    "ACC", "PGC", "SGC",
    "L Amygdala", "R Amygdala",
    "L ALIC", "R ALIC"
]


Z_DEPTH = 0  # Estimated depth of the face relative to brain (mm)

# ------------------------
# Utility Functions
# ------------------------

def promote_to_3d(points_2d, z_value=-50):
    return np.hstack([points_2d, np.full((points_2d.shape[0], 1), z_value)])

def estimate_similarity_transform(template_pts, subject_pts):
    template_mean = template_pts.mean(axis=0)
    subject_mean = subject_pts.mean(axis=0)
    A = template_pts - template_mean
    B = subject_pts - subject_mean
    H = A.T @ B
    U, _, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T
    s = np.trace(H @ R.T) / np.trace(A.T @ A)
    t = subject_mean - s * R @ template_mean
    return s, R, t

def apply_transform(points, s, R, t):
    return s * (points @ R.T) + t

def invert_transform(points, s, R, t):
    R_inv = R.T
    s_inv = 1.0 / s
    return s_inv * ((points - t) @ R_inv)

def reprojection_error(original, recovered):
    return np.linalg.norm(original - recovered, axis=1)

# ------------------------
# Plotting
# ------------------------

def plot_subject_space(fiducials, targets):
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
        x=fiducials[:, 0], y=fiducials[:, 1], z=fiducials[:, 2],
        mode='markers+text',
        marker=dict(size=5, color='blue'),
        text=FIDUCIAL_NAMES,
        name='Facial Fiducials'
    ))

    fig.add_trace(go.Scatter3d(
        x=targets[:, 0], y=targets[:, 1], z=targets[:, 2],
        mode='markers+text',
        marker=dict(size=5, color='red'),
        text=TARGET_NAMES,
        name='Estimated Brain Targets'
    ))

    fig.update_layout(
        title="MRI-Free Fiducials and Brain Targets (Subject Space)",
        scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='data'),
        margin=dict(l=0, r=0, t=40, b=0)
    )
    fig.show()

# ------------------------
# Main Pipeline
# ------------------------

def run_pipeline(subject_fiducials_2d):
    subject_fiducials_3d = promote_to_3d(subject_fiducials_2d, Z_DEPTH)
    s, R, t = estimate_similarity_transform(TEMPLATE_FIDUCIALS, subject_fiducials_3d)
    subject_targets = apply_transform(TEMPLATE_TARGETS, s, R, t)
    fiducials_mni = invert_transform(subject_fiducials_3d, s, R, t)
    template_recovered = invert_transform(subject_targets, s, R, t)

    # Optional: print reprojection error
    err = reprojection_error(TEMPLATE_TARGETS, template_recovered)
    print("Target reprojection error (mm):", np.round(err, 2))

    plot_subject_space(subject_fiducials_3d, subject_targets)

    return {
        "s": s, "R": R, "t": t,
        "subject_targets": subject_targets,
        "fiducials_mni": fiducials_mni,
        "template_recovered": template_recovered
    }

results = run_pipeline(subject_fiducials_2d)

fiducials_mni = results["fiducials_mni"]
template_targets = TEMPLATE_TARGETS  # already defined in the config

template_targets = results["template_recovered"]

In [ ]:
from google.colab import files
uploaded = files.upload()

filename = next(iter(uploaded))  # gets your .nii file name


In [ ]:
# Path to your NIfTI file
nii_file = 'icbm_avg_152_t1_tal_nlin_symmetric_VI.nii'

# Load the image
img = nib.load(nii_file)
data = img.get_fdata()
affine = img.affine


# Extract brain surface with marching cubes
level = 30  # Tune this based on your volume

verts, faces, normals, _ = measure.marching_cubes(data, level=level)

# Convert to MNI space using affine
verts_h = np.hstack([verts, np.ones((verts.shape[0], 1))])
verts_mni = verts_h @ affine.T

In [ ]:
# Plot targets with brain

fig = go.Figure()

# Brain mesh
fig.add_trace(go.Mesh3d(
    x=verts_mni[:, 0], y=verts_mni[:, 1], z=verts_mni[:, 2],
    i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
    color='lightgrey', opacity=0.2, name='MNI Brain'
))

# Fiducials
fig.add_trace(go.Scatter3d(
    x=fiducials_mni[:, 0],
    y=fiducials_mni[:, 1],
    z=fiducials_mni[:, 2],
    mode='markers+text',
    marker=dict(size=5, color='blue'),
    text=FIDUCIAL_NAMES,
    name='Facial Fiducials'
))

# Deep brain targets (ACC etc.)
fig.add_trace(go.Scatter3d(
    x=template_targets[:, 0],
    y=template_targets[:, 1],
    z=template_targets[:, 2],
    mode='markers+text',
    marker=dict(size=5, color='red'),
    text=TARGET_NAMES,
    name='Deep Brain Targets'
))

fig.update_layout(
    title="MRI-Free Fiducials + Deep Targets Overlaid on MNI Brain",
    scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z', aspectmode='data'),
    margin=dict(l=0, r=0, t=40, b=0)
)

fig.show()
